# Data Quality Check - Master Aadhaar Data

This notebook performs comprehensive data quality checks on the `master_aadhaar_data_final_cleaned.csv` file to identify:
- Null values
- Duplicate rows
- State name mismatches/inconsistencies
- District name mismatches/inconsistencies
- Pincode issues
- Data type validation

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# For better display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load the Dataset

In [4]:
# Load the dataset
df = pd.read_csv('master_aadhaar_data_fully_cleaned.csv')

print(f"Dataset Shape: {df.shape}")
print(f"Total Rows: {df.shape[0]:,}")
print(f"Total Columns: {df.shape[1]}")
print("\nFirst few rows:")
df.head()

Dataset Shape: (2307730, 20)
Total Rows: 2,307,730
Total Columns: 20

First few rows:


,date,state,district,pincode,month_name,day_name,is_weekend,age_0_5,age_5_17,age_18_greater,bio_age_5_17,bio_age_18_greater,demo_age_5_17,demo_age_18_greater,total_enrolments,total_biometric_updates,total_demographic_updates,total_updates,overall_activity,update_to_enrolment_ratio
0,2025-03-01,Andaman and Nicobar Islands,Andamans,744101,March,Saturday,1,0,0,0,16,193,0,0,0,209,0,209,209,2090.0
1,2025-03-01,Andaman and Nicobar Islands,Nicobar,744301,March,Saturday,1,0,0,0,101,48,16,180,0,149,196,345,345,3450.0
2,2025-03-01,Andaman and Nicobar Islands,Nicobar,744302,March,Saturday,1,0,0,0,15,12,0,0,0,27,0,27,27,270.0
3,2025-03-01,Andaman and Nicobar Islands,Nicobar,744303,March,Saturday,1,0,0,0,46,27,0,0,0,73,0,73,73,730.0
4,2025-03-01,Andaman and Nicobar Islands,Nicobar,744304,March,Saturday,1,0,0,0,16,14,0,0,0,30,0,30,30,300.0


## 2. Basic Dataset Information

In [3]:
# Dataset info
print("Dataset Information:")
print("="*80)
df.info()

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2330372 entries, 0 to 2330371
Data columns (total 20 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   date                       object 
 1   state                      object 
 2   district                   object 
 3   pincode                    int64  
 4   age_0_5                    int64  
 5   age_5_17                   int64  
 6   age_18_greater             int64  
 7   bio_age_5_17               int64  
 8   bio_age_18_greater         int64  
 9   demo_age_5_17              int64  
 10  demo_age_18_greater        int64  
 11  total_enrolments           int64  
 12  total_biometric_updates    int64  
 13  total_demographic_updates  int64  
 14  total_updates              int64  
 15  overall_activity           int64  
 16  update_to_enrolment_ratio  float64
 17  month_name                 object 
 18  day_name                   object 
 19  is_weekend           

In [4]:
# Column names
print("\nColumn Names:")
print("="*80)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")


Column Names:
 1. date
 2. state
 3. district
 4. pincode
 5. age_0_5
 6. age_5_17
 7. age_18_greater
 8. bio_age_5_17
 9. bio_age_18_greater
10. demo_age_5_17
11. demo_age_18_greater
12. total_enrolments
13. total_biometric_updates
14. total_demographic_updates
15. total_updates
16. overall_activity
17. update_to_enrolment_ratio
18. month_name
19. day_name
20. is_weekend


## 3. Check for Null Values

In [5]:
# Check null values
print("\n" + "="*80)
print("NULL VALUE ANALYSIS")
print("="*80)

null_counts = df.isnull().sum()
null_percentages = (df.isnull().sum() / len(df)) * 100

null_df = pd.DataFrame({
    'Column': null_counts.index,
    'Null Count': null_counts.values,
    'Null Percentage': null_percentages.values
})

null_df = null_df[null_df['Null Count'] > 0].sort_values('Null Count', ascending=False)

if len(null_df) > 0:
    print("\n⚠️ COLUMNS WITH NULL VALUES:")
    print(null_df.to_string(index=False))
else:
    print("\n✓ No null values found in the dataset!")

# Total nulls
total_nulls = df.isnull().sum().sum()
print(f"\nTotal Null Values: {total_nulls:,}")


NULL VALUE ANALYSIS

✓ No null values found in the dataset!

Total Null Values: 0


## 4. Check for Duplicate Rows

In [4]:
print("\n" + "="*80)
print("DUPLICATE ROWS ANALYSIS")
print("="*80)

# Check for complete duplicates
duplicate_rows = df.duplicated().sum()
print(f"\nTotal Duplicate Rows (exact duplicates): {duplicate_rows:,}")

if duplicate_rows > 0:
    print(f"Percentage of duplicates: {(duplicate_rows/len(df)*100):.2f}%")
    print("\n⚠️ Sample of duplicate rows:")
    display(df[df.duplicated(keep=False)].head(10))
else:
    print("✓ No exact duplicate rows found!")

# Check for duplicates based on key columns (date, state, district, pincode)
key_columns = ['date', 'state', 'district', 'pincode']
duplicate_keys = df.duplicated(subset=key_columns).sum()
print(f"\nDuplicate rows based on key columns {key_columns}: {duplicate_keys:,}")

if duplicate_keys > 0:
    print(f"Percentage of key duplicates: {(duplicate_keys/len(df)*100):.2f}%")
    print("\n⚠️ Sample of duplicate key combinations:")
    display(df[df.duplicated(subset=key_columns, keep=False)].sort_values(key_columns).head(10))
else:
    print("✓ No duplicate key combinations found!")


DUPLICATE ROWS ANALYSIS

Total Duplicate Rows (exact duplicates): 0
✓ No exact duplicate rows found!

Duplicate rows based on key columns ['date', 'state', 'district', 'pincode']: 0
✓ No duplicate key combinations found!


## 5. State Names Analysis

In [5]:
print("\n" + "="*80)
print("STATE NAMES ANALYSIS")
print("="*80)

# Get unique states
unique_states = df['state'].unique()
print(f"\nTotal Unique States: {len(unique_states)}")
print(f"Expected States/UTs in India: ~36")

# Display all unique states
print("\nAll Unique State Names:")
for i, state in enumerate(sorted(unique_states), 1):
    state_count = df[df['state'] == state].shape[0]
    print(f"{i:3}. {state:<50} (Records: {state_count:,})")


STATE NAMES ANALYSIS

Total Unique States: 36
Expected States/UTs in India: ~36

All Unique State Names:
  1. Andaman and Nicobar Islands                        (Records: 1,975)
  2. Andhra Pradesh                                     (Records: 220,344)
  3. Arunachal Pradesh                                  (Records: 5,351)
  4. Assam                                              (Records: 64,630)
  5. Bihar                                              (Records: 104,072)
  6. Chandigarh                                         (Records: 2,118)
  7. Chhattisgarh                                       (Records: 38,863)
  8. Dadra and Nagar Haveli and Daman and Diu           (Records: 1,043)
  9. Delhi                                              (Records: 11,295)
 10. Goa                                                (Records: 6,841)
 11. Gujarat                                            (Records: 110,943)
 12. Haryana                                            (Records: 31,494)
 13. Him

In [8]:
# Check for potential state name issues
print("\n" + "="*80)
print("POTENTIAL STATE NAME ISSUES")
print("="*80)

# Normalize state names for comparison
df['state_normalized'] = df['state'].str.strip().str.lower()

# Check for case inconsistencies
state_variations = df.groupby('state_normalized')['state'].unique()
inconsistent_states = state_variations[state_variations.apply(len) > 1]

if len(inconsistent_states) > 0:
    print("\n⚠️ State names with case/whitespace inconsistencies:")
    for normalized, variations in inconsistent_states.items():
        print(f"\nNormalized: '{normalized}'")
        for var in variations:
            count = df[df['state'] == var].shape[0]
            print(f"  - '{var}' (Records: {count:,})")
else:
    print("✓ No case/whitespace inconsistencies found in state names!")

# Remove temporary column
df.drop('state_normalized', axis=1, inplace=True)


POTENTIAL STATE NAME ISSUES
✓ No case/whitespace inconsistencies found in state names!


In [9]:
# Check for similar state names (potential typos)
print("\n" + "="*80)
print("CHECKING FOR SIMILAR STATE NAMES (POTENTIAL TYPOS)")
print("="*80)

from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Find similar state names
similar_states = []
states_list = sorted(unique_states)

for i in range(len(states_list)):
    for j in range(i+1, len(states_list)):
        sim = similarity(states_list[i], states_list[j])
        if 0.6 < sim < 1.0:  # Similar but not identical
            similar_states.append((states_list[i], states_list[j], sim))

if similar_states:
    print("\n⚠️ Potentially similar state names found:")
    for state1, state2, sim in sorted(similar_states, key=lambda x: x[2], reverse=True):
        count1 = df[df['state'] == state1].shape[0]
        count2 = df[df['state'] == state2].shape[0]
        print(f"\nSimilarity: {sim:.2%}")
        print(f"  1. '{state1}' (Records: {count1:,})")
        print(f"  2. '{state2}' (Records: {count2:,})")
else:
    print("\n✓ No similar state names found!")


CHECKING FOR SIMILAR STATE NAMES (POTENTIAL TYPOS)

⚠️ Potentially similar state names found:

Similarity: 85.71%
  1. 'Andhra Pradesh' (Records: 220,394)
  2. 'Madhya Pradesh' (Records: 86,008)

Similarity: 80.00%
  1. 'Himachal Pradesh' (Records: 36,340)
  2. 'Madhya Pradesh' (Records: 86,008)

Similarity: 78.79%
  1. 'Arunachal Pradesh' (Records: 5,351)
  2. 'Himachal Pradesh' (Records: 36,340)

Similarity: 77.42%
  1. 'Andhra Pradesh' (Records: 220,394)
  2. 'Arunachal Pradesh' (Records: 5,351)

Similarity: 74.07%
  1. 'Andhra Pradesh' (Records: 220,394)
  2. 'Uttar Pradesh' (Records: 190,896)

Similarity: 73.33%
  1. 'Andhra Pradesh' (Records: 220,394)
  2. 'Himachal Pradesh' (Records: 36,340)

Similarity: 72.73%
  1. 'Chandigarh' (Records: 2,118)
  2. 'Chhattisgarh' (Records: 38,870)

Similarity: 70.00%
  1. 'Jharkhand' (Records: 45,629)
  2. 'Uttarakhand' (Records: 26,922)

Similarity: 66.67%
  1. 'Arunachal Pradesh' (Records: 5,351)
  2. 'Uttar Pradesh' (Records: 190,896)

Sim

## 6. District Names Analysis

In [10]:
print("\n" + "="*80)
print("DISTRICT NAMES ANALYSIS")
print("="*80)

# Get unique districts
unique_districts = df['district'].unique()
print(f"\nTotal Unique Districts: {len(unique_districts)}")
print(f"Expected Districts in India: ~750")

# Districts per state
districts_per_state = df.groupby('state')['district'].nunique().sort_values(ascending=False)
print("\nDistricts per State (Top 10):")
print(districts_per_state.head(10))


DISTRICT NAMES ANALYSIS

Total Unique Districts: 1001
Expected Districts in India: ~750

Districts per State (Top 10):
state
Uttar Pradesh     96
Madhya Pradesh    61
Karnataka         56
Maharashtra       55
West Bengal       55
Bihar             48
Andhra Pradesh    48
Tamil Nadu        47
Odisha            46
Rajasthan         46
Name: district, dtype: int64


In [11]:
# Check for districts appearing in multiple states
print("\n" + "="*80)
print("DISTRICTS APPEARING IN MULTIPLE STATES")
print("="*80)

district_state_mapping = df.groupby('district')['state'].unique()
multi_state_districts = district_state_mapping[district_state_mapping.apply(len) > 1]

if len(multi_state_districts) > 0:
    print(f"\n⚠️ {len(multi_state_districts)} districts found in multiple states:")
    for district, states in multi_state_districts.items():
        print(f"\nDistrict: '{district}'")
        for state in states:
            count = df[(df['district'] == district) & (df['state'] == state)].shape[0]
            print(f"  - {state} (Records: {count:,})")
else:
    print("\n✓ No districts appear in multiple states!")


DISTRICTS APPEARING IN MULTIPLE STATES

⚠️ 23 districts found in multiple states:

District: 'Adilabad'
  - Andhra Pradesh (Records: 3,813)
  - Telangana (Records: 4,164)

District: 'Aurangabad'
  - Bihar (Records: 2,707)
  - Maharashtra (Records: 4,023)

District: 'Balrampur'
  - Chhattisgarh (Records: 722)
  - Uttar Pradesh (Records: 1,415)

District: 'Bijapur'
  - Chhattisgarh (Records: 582)
  - Karnataka (Records: 4,025)

District: 'Bilaspur'
  - Chhattisgarh (Records: 2,330)
  - Himachal Pradesh (Records: 2,158)

District: 'Cuddalore'
  - Tamil Nadu (Records: 6,431)
  - Puducherry (Records: 5)

District: 'Hamirpur'
  - Himachal Pradesh (Records: 3,682)
  - Uttar Pradesh (Records: 1,417)

District: 'Hyderabad'
  - Andhra Pradesh (Records: 5,219)
  - Telangana (Records: 5,973)

District: 'K.V. Rangareddy'
  - Andhra Pradesh (Records: 358)
  - Telangana (Records: 8,687)

District: 'Kamrup'
  - Assam (Records: 3,788)
  - Meghalaya (Records: 1)

District: 'Kargil'
  - Jammu and Kashmi

In [12]:
# Check for case inconsistencies in district names
print("\n" + "="*80)
print("DISTRICT NAME INCONSISTENCIES")
print("="*80)

# Group by state and check for district inconsistencies
inconsistent_districts_found = False

for state in df['state'].unique():
    state_df = df[df['state'] == state]
    state_df_normalized = state_df.copy()
    state_df_normalized['district_normalized'] = state_df_normalized['district'].str.strip().str.lower()
    
    district_variations = state_df_normalized.groupby('district_normalized')['district'].unique()
    inconsistent = district_variations[district_variations.apply(len) > 1]
    
    if len(inconsistent) > 0:
        if not inconsistent_districts_found:
            print("\n⚠️ District names with case/whitespace inconsistencies:")
            inconsistent_districts_found = True
        
        print(f"\nState: {state}")
        for normalized, variations in inconsistent.items():
            print(f"  Normalized: '{normalized}'")
            for var in variations:
                count = state_df[state_df['district'] == var].shape[0]
                print(f"    - '{var}' (Records: {count:,})")

if not inconsistent_districts_found:
    print("\n✓ No case/whitespace inconsistencies found in district names!")


DISTRICT NAME INCONSISTENCIES

✓ No case/whitespace inconsistencies found in district names!


## 7. Pincode Analysis

In [6]:
print("\n" + "="*80)
print("PINCODE ANALYSIS")
print("="*80)

# Basic pincode statistics
unique_pincodes = df['pincode'].unique()
print(f"\nTotal Unique Pincodes: {len(unique_pincodes):,}")
print(f"Expected Pincodes in India: ~19,000+")

# Check data type
print(f"\nPincode Data Type: {df['pincode'].dtype}")

# Basic statistics
print("\nPincode Statistics:")
print(df['pincode'].describe())


PINCODE ANALYSIS

Total Unique Pincodes: 19,814
Expected Pincodes in India: ~19,000+

Pincode Data Type: int64

Pincode Statistics:
count    2.307730e+06
mean     5.235432e+05
std      1.973077e+05
min      1.100010e+05
25%      3.931300e+05
50%      5.232130e+05
75%      6.891150e+05
max      8.554560e+05
Name: pincode, dtype: float64


In [7]:
# Check for invalid pincodes
print("\n" + "="*80)
print("PINCODE VALIDATION")
print("="*80)

# Convert to string for validation
df['pincode_str'] = df['pincode'].astype(str)

# Check pincode length (should be 6 digits)
invalid_length = df[df['pincode_str'].str.len() != 6]
print(f"\nPincodes with invalid length (not 6 digits): {len(invalid_length):,}")
if len(invalid_length) > 0:
    print("\n⚠️ Sample of invalid length pincodes:")
    print(invalid_length[['state', 'district', 'pincode']].head(10))

# Check for non-numeric pincodes
non_numeric = df[~df['pincode_str'].str.isdigit()]
print(f"\nNon-numeric pincodes: {len(non_numeric):,}")
if len(non_numeric) > 0:
    print("\n⚠️ Sample of non-numeric pincodes:")
    print(non_numeric[['state', 'district', 'pincode']].head(10))

# Check pincode ranges by first digit (Indian pincodes start from 1-9)
df['pincode_first_digit'] = df['pincode_str'].str[0]
print("\nPincode distribution by first digit:")
print(df['pincode_first_digit'].value_counts().sort_index())

# Check for pincodes starting with 0
pincodes_start_zero = df[df['pincode_str'].str.startswith('0')]
if len(pincodes_start_zero) > 0:
    print(f"\n⚠️ Pincodes starting with 0: {len(pincodes_start_zero):,}")
    print(pincodes_start_zero[['state', 'district', 'pincode']].head(10))

# Clean up temp columns
df.drop(['pincode_str', 'pincode_first_digit'], axis=1, inplace=True)


PINCODE VALIDATION

Pincodes with invalid length (not 6 digits): 0

Non-numeric pincodes: 0

Pincode distribution by first digit:
pincode_first_digit
1    163417
2    217712
3    213190
4    313478
5    501279
6    354509
7    395502
8    148643
Name: count, dtype: int64


In [8]:
# Check for pincodes appearing in multiple states
print("\n" + "="*80)
print("PINCODES APPEARING IN MULTIPLE STATES")
print("="*80)

pincode_state_mapping = df.groupby('pincode')['state'].unique()
multi_state_pincodes = pincode_state_mapping[pincode_state_mapping.apply(len) > 1]

if len(multi_state_pincodes) > 0:
    print(f"\n⚠️ {len(multi_state_pincodes)} pincodes found in multiple states:")
    # Show first 10
    for pincode, states in list(multi_state_pincodes.items())[:10]:
        print(f"\nPincode: {pincode}")
        for state in states:
            count = df[(df['pincode'] == pincode) & (df['state'] == state)].shape[0]
            districts = df[(df['pincode'] == pincode) & (df['state'] == state)]['district'].unique()
            print(f"  - {state} (Districts: {', '.join(districts)}, Records: {count:,})")
    
    if len(multi_state_pincodes) > 10:
        print(f"\n... and {len(multi_state_pincodes) - 10} more")
else:
    print("\n✓ No pincodes appear in multiple states!")


PINCODES APPEARING IN MULTIPLE STATES

⚠️ 705 pincodes found in multiple states:

Pincode: 140308
  - Punjab (Districts: Sas Nagar (Mohali), S.A.S Nagar(Mohali), Rupnagar, Records: 159)
  - Chandigarh (Districts: Chandigarh, Records: 2)

Pincode: 140603
  - Punjab (Districts: Sas Nagar (Mohali), S.A.S Nagar(Mohali), Rupnagar, Patiala, Records: 186)
  - Chandigarh (Districts: Mohali, Records: 9)

Pincode: 140901
  - Punjab (Districts: Sas Nagar (Mohali), S.A.S Nagar(Mohali), Rupnagar, Records: 171)
  - Chandigarh (Districts: Chandigarh, Records: 20)

Pincode: 160003
  - Chandigarh (Districts: Chandigarh, Rupnagar, Records: 95)
  - Punjab (Districts: Sas Nagar (Mohali), S.A.S Nagar(Mohali), Records: 12)

Pincode: 160014
  - Chandigarh (Districts: Chandigarh, Rupnagar, Records: 122)
  - Punjab (Districts: Sas Nagar (Mohali), S.A.S Nagar(Mohali), Rupnagar, Records: 138)

Pincode: 160055
  - Chandigarh (Districts: Chandigarh, Rupnagar, Records: 104)
  - Punjab (Districts: Sas Nagar (Mohali

In [9]:
# Check for pincodes appearing in multiple districts within the same state
print("\n" + "="*80)
print("PINCODES IN MULTIPLE DISTRICTS (SAME STATE)")
print("="*80)

multi_district_pincodes = []

for state in df['state'].unique():
    state_df = df[df['state'] == state]
    pincode_districts = state_df.groupby('pincode')['district'].unique()
    multi_districts = pincode_districts[pincode_districts.apply(len) > 1]
    
    if len(multi_districts) > 0:
        multi_district_pincodes.append((state, multi_districts))

if multi_district_pincodes:
    print("\n⚠️ Pincodes appearing in multiple districts within the same state:")
    for state, pincodes in multi_district_pincodes[:5]:  # Show first 5 states
        print(f"\nState: {state}")
        for pincode, districts in list(pincodes.items())[:3]:  # Show first 3 pincodes per state
            print(f"  Pincode {pincode}: {', '.join(districts)}")
        if len(pincodes) > 3:
            print(f"  ... and {len(pincodes) - 3} more pincodes")
else:
    print("\n✓ No pincodes appear in multiple districts within the same state!")


PINCODES IN MULTIPLE DISTRICTS (SAME STATE)

⚠️ Pincodes appearing in multiple districts within the same state:

State: Andaman and Nicobar Islands
  Pincode 744101: Andamans, South Andaman
  Pincode 744102: South Andaman, Andamans
  Pincode 744103: South Andaman, Andamans
  ... and 11 more pincodes

State: Andhra Pradesh
  Pincode 500005: Hyderabad, Rangareddi, K.V.Rangareddy, K.V. Rangareddy
  Pincode 500008: Hyderabad, Rangareddi, K.V.Rangareddy
  Pincode 500014: Rangareddi, K.V.Rangareddy, Hyderabad, K.V. Rangareddy
  ... and 1000 more pincodes

State: Arunachal Pradesh
  Pincode 786629: Tirap, Longding
  Pincode 786630: Tirap, Longding
  Pincode 786631: Longding, Tirap
  ... and 14 more pincodes

State: Assam
  Pincode 781001: Kamrup Metro, Kamrup
  Pincode 781003: Kamrup Metro, Kamrup
  Pincode 781004: Kamrup Metro, Kamrup
  ... and 294 more pincodes

State: Bihar
  Pincode 801304: Nalanda, Patna
  Pincode 801305: Nalanda, Patna
  Pincode 802112: Bhojpur, Buxar
  ... and 361 mor

## 8. Data Type Validation

In [10]:
print("\n" + "="*80)
print("DATA TYPE VALIDATION")
print("="*80)

# Check numeric columns
numeric_columns = [
    'age_0_5', 'age_5_17', 'age_18_greater',
    'bio_age_5_17', 'bio_age_18_greater',
    'demo_age_5_17', 'demo_age_18_greater',
    'total_enrolments', 'total_biometric_updates', 'total_demographic_updates',
    'total_updates', 'overall_activity', 'update_to_enrolment_ratio'
]

print("\nNumeric Column Data Types:")
for col in numeric_columns:
    if col in df.columns:
        print(f"{col:<35} : {df[col].dtype}")


DATA TYPE VALIDATION

Numeric Column Data Types:
age_0_5                             : int64
age_5_17                            : int64
age_18_greater                      : int64
bio_age_5_17                        : int64
bio_age_18_greater                  : int64
demo_age_5_17                       : int64
demo_age_18_greater                 : int64
total_enrolments                    : int64
total_biometric_updates             : int64
total_demographic_updates           : int64
total_updates                       : int64
overall_activity                    : int64
update_to_enrolment_ratio           : float64


In [11]:
# Check for negative values in numeric columns (where they shouldn't exist)
print("\n" + "="*80)
print("NEGATIVE VALUES CHECK")
print("="*80)

negative_issues = False
for col in numeric_columns:
    if col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                print(f"\n⚠️ Column '{col}' has {negative_count:,} negative values")
                negative_issues = True

if not negative_issues:
    print("\n✓ No negative values found in numeric columns!")


NEGATIVE VALUES CHECK

✓ No negative values found in numeric columns!


In [12]:
# Check date column
print("\n" + "="*80)
print("DATE COLUMN VALIDATION")
print("="*80)

print(f"\nDate column data type: {df['date'].dtype}")

# Try to parse dates
try:
    df['date_parsed'] = pd.to_datetime(df['date'])
    print("✓ Date column can be parsed as datetime")
    print(f"\nDate range: {df['date_parsed'].min()} to {df['date_parsed'].max()}")
    print(f"Total unique dates: {df['date_parsed'].nunique():,}")
    df.drop('date_parsed', axis=1, inplace=True)
except Exception as e:
    print(f"\n⚠️ Error parsing date column: {e}")


DATE COLUMN VALIDATION

Date column data type: object
✓ Date column can be parsed as datetime

Date range: 2025-03-01 00:00:00 to 2025-12-31 00:00:00
Total unique dates: 115


## 9. Additional Data Quality Checks

In [13]:
# Check for whitespace issues in string columns
print("\n" + "="*80)
print("WHITESPACE ISSUES CHECK")
print("="*80)

string_columns = ['state', 'district', 'month_name', 'day_name']
whitespace_issues = False

for col in string_columns:
    if col in df.columns:
        # Check for leading/trailing whitespace
        leading_space = df[df[col].str.startswith(' ')].shape[0]
        trailing_space = df[df[col].str.endswith(' ')].shape[0]
        
        if leading_space > 0 or trailing_space > 0:
            print(f"\n⚠️ Column '{col}':")
            if leading_space > 0:
                print(f"  - Leading whitespace: {leading_space:,} rows")
            if trailing_space > 0:
                print(f"  - Trailing whitespace: {trailing_space:,} rows")
            whitespace_issues = True

if not whitespace_issues:
    print("\n✓ No whitespace issues found in string columns!")


WHITESPACE ISSUES CHECK

✓ No whitespace issues found in string columns!


In [14]:
# Check for empty strings
print("\n" + "="*80)
print("EMPTY STRING CHECK")
print("="*80)

empty_string_issues = False
for col in string_columns:
    if col in df.columns:
        empty_count = (df[col] == '').sum()
        if empty_count > 0:
            print(f"\n⚠️ Column '{col}' has {empty_count:,} empty strings")
            empty_string_issues = True

if not empty_string_issues:
    print("\n✓ No empty strings found in string columns!")


EMPTY STRING CHECK

✓ No empty strings found in string columns!


## 10. Monthly Data Analysis

In [5]:
# Check records per month
print("\n" + "="*80)
print("MONTHLY DATA ANALYSIS")
print("="*80)

# Ensure date is datetime for accurate sorting
df['date_dt'] = pd.to_datetime(df['date'])
df['month_num'] = df['date_dt'].dt.month
df['year'] = df['date_dt'].dt.year

# Records per month
monthly_stats = df.groupby(['year', 'month_num', 'month_name']).agg({
    'date': 'nunique',
    'state': 'count'
}).rename(columns={'date': 'unique_days', 'state': 'record_count'}).reset_index()

monthly_stats = monthly_stats.sort_values(['year', 'month_num'])

print("\nMonthly Distribution:")
print("="*30)
print(monthly_stats[['year', 'month_name', 'unique_days', 'record_count']].to_string(index=False))

# Check for missing months in the range
all_months_range = pd.date_range(start=df['date_dt'].min(), end=df['date_dt'].max(), freq='MS').strftime('%B').tolist()
present_months = df['month_name'].unique().tolist()
missing_months = [m for m in all_months_range if m not in present_months]

if missing_months:
    print(f"\n⚠️ Missing months between {df['date_dt'].min().strftime('%B %Y')} and {df['date_dt'].max().strftime('%B %Y')}:")
    print(", ".join(missing_months))
else:
    print(f"\n✓ All months between {df['date_dt'].min().strftime('%B %Y')} and {df['date_dt'].max().strftime('%B %Y')} are present!")

# Check for consistency in days per month
print("\nObservations:")
for _, row in monthly_stats.iterrows():
    if row['unique_days'] < 28:
        print(f"  - {row['month_name']} {row['year']} has only {row['unique_days']} days of data.")

# Clean up temp columns
df.drop(['date_dt', 'month_num', 'year'], axis=1, inplace=True)


MONTHLY DATA ANALYSIS

Monthly Distribution:
 year month_name  unique_days  record_count
 2025      March           10         22346
 2025      April            1         21528
 2025        May            1         21919
 2025       June            1         22068
 2025       July            1         22314
 2025  September           30        551365
 2025    October           18        394907
 2025   November           24        566711
 2025   December           29        684572

⚠️ Missing months between March 2025 and December 2025:
August

Observations:
  - March 2025 has only 10 days of data.
  - April 2025 has only 1 days of data.
  - May 2025 has only 1 days of data.
  - June 2025 has only 1 days of data.
  - July 2025 has only 1 days of data.
  - October 2025 has only 18 days of data.
  - November 2025 has only 24 days of data.


## 11. Summary Report

In [15]:
print("\n" + "="*80)
print("SUMMARY REPORT")
print("="*80)

print(f"\nDataset Overview:")
print(f"  - Total Rows: {df.shape[0]:,}")
print(f"  - Total Columns: {df.shape[1]}")
print(f"  - Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nData Quality Metrics:")
print(f"  - Total Null Values: {df.isnull().sum().sum():,}")
print(f"  - Duplicate Rows: {df.duplicated().sum():,}")
print(f"  - Unique States: {df['state'].nunique()}")
print(f"  - Unique Districts: {df['district'].nunique()}")
print(f"  - Unique Pincodes: {df['pincode'].nunique():,}")
print(f"  - Unique Dates: {df['date'].nunique():,}")

print(f"\nData Ranges:")
print(f"  - Total Enrolments: {df['total_enrolments'].min():,.0f} to {df['total_enrolments'].max():,.0f}")
print(f"  - Total Updates: {df['total_updates'].min():,.0f} to {df['total_updates'].max():,.0f}")
print(f"  - Overall Activity: {df['overall_activity'].min():,.0f} to {df['overall_activity'].max():,.0f}")

print("\n" + "="*80)
print("DATA QUALITY CHECK COMPLETED!")
print("="*80)


SUMMARY REPORT

Dataset Overview:
  - Total Rows: 2,307,730
  - Total Columns: 20
  - Memory Usage: 899.92 MB

Data Quality Metrics:
  - Total Null Values: 0
  - Duplicate Rows: 0
  - Unique States: 36
  - Unique Districts: 1001
  - Unique Pincodes: 19,814
  - Unique Dates: 115

Data Ranges:
  - Total Enrolments: 0 to 3,965
  - Total Updates: 0 to 28,298
  - Overall Activity: 0 to 30,000

DATA QUALITY CHECK COMPLETED!
